# Whois Domain Lookup

In the following code we use the 'ipapi.co' API, which is an API used to fetch GeoIP data, to retrieve information about the domain. The 'ipapi.co' API blocks the requests from the Python request library to prevent automated scraping and abuse. To get around this I used `headers` to indentify as a user accessing the API from the Chrome browser on a Windows 10 computer. (https://ipapi.co/)

While the following code fetched GeoIP data, you can use the python-whois library to fetch actual WHOIS data.

In [ ]:
import socket
import requests

def get_domain_info(domain):
    try:
        # Get IP address (active but low-risk)
        ip = socket.gethostbyname(domain)
        print(f"IP Address: {ip}")

        # Mimics Chrome browser from a Windows 10 computer
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

        # Get public WHOIS-like info (passive, using a free API)
        response = requests.get(f"https://ipapi.co/{ip}/json/", headers=headers)
        if response.status_code == 200:
            data = response.json()
            print(f"Organization: {data.get('org', 'Unknown')}")
            print(f"City: {data.get('city', 'Unknown')}")
            print(f"Country: {data.get('country_name', 'Unknown')}")
        else:
            print("Could not fetch WHOIS data.")
    except Exception as e:
        print(f"Error: {e}")

# Example: Use a public domain (never use without permission!)
get_domain_info("python.com")

IP Address: 185.158.133.1
Organization: CLOUDFLARENET
City: Frankfurt am Main
Country: Germany


# Types of Penetration Tests

We use the following code to simulate a simple black box recon on 'python.org'.

In [6]:
import requests

def black_box_recon(url):
    try:
        response = requests.head(url)
        print("Black Box Findings:")
        print(f"Server: {response.headers.get('Server', 'Unknown')}")
        print(f"Content-Type: {response.headers.get('Content-Type', 'Unknown')}")
    except Exception as e:
        print(f"Error: {e}")

url = "http://python.org"
known_info = {"server": "Apache 2.4", "vulns": "Check CVE-2021-1234"}
black_box_recon(url)

Black Box Findings:
Server: Varnish
Content-Type: Unknown


# Penetration Testing Methodology and Tools

Following code scans for open ports on `localhost`.

In [19]:
import socket

def scan_ports(host, ports):
    open_ports = []
    for port in ports:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex((host, port))
        if result == 0:
            open_ports.append(port)
        sock.close()
    return open_ports

host = "127.0.0.1"
ports = [80, 443, 22, 8080]
open_ports = scan_ports(host, ports)
print(f"Open ports on {host}: {open_ports}")

Open ports on 127.0.0.1: []


There are no open ports on `localhost`.

# Using Nmap in Python

In [ ]:
# pip install python-nmap

  Using cached python_nmap-0.7.1-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [4]:
import nmap

def nmap_scan(host, port_range='1-1024'):
    nm = nmap.PortScanner()
    try:
        nm.scan(host, port_range, arguments='-sV') # -sV for service version detection
        for host in nm.all_hosts():
            print(f"Host: {host} ({nm[host].hostname()})")
            print(f"State: {nm[host].state()}")
            for proto in nm[host].all_protocols():
                print(f"Protocol: {proto}")
                lport = nm[host][proto].keys()
                for port in sorted(lport):
                    service = nm[host][proto][port]
                    print(f"Port: {port}\tState: {service['state']}\tService: {service.get('name', 'unknown')} {service.get('version', '')}")
                    
    except Exception as e:
        print(f"Error: {e}")

# Example: Scan localhost
nmap_scan('127.0.0.1', '1-10')

Host: 127.0.0.1 (localhost)
State: up
Protocol: tcp
Port: 1	State: closed	Service: tcpmux 
Port: 2	State: closed	Service: compressnet 
Port: 3	State: closed	Service: compressnet 
Port: 4	State: closed	Service:  
Port: 5	State: closed	Service: rje 
Port: 6	State: closed	Service:  
Port: 7	State: closed	Service: echo 
Port: 8	State: closed	Service:  
Port: 9	State: closed	Service: discard 
Port: 10	State: closed	Service:  


In [5]:
nmap_scan('127.0.0.1', '1-1024')

Host: 127.0.0.1 (localhost)
State: up
Protocol: tcp
Port: 135	State: open	Service: msrpc 
Port: 137	State: filtered	Service: netbios-ns 
Port: 445	State: open	Service: microsoft-ds 


After scanning 1024 ports, nmap only printed the 3 ports of interest:
- **Port 135:** This is the RPC Endpoint Mapper service. It lets other systems know what services are available on a machine and which ports they can be found at. (https://www.helpwire.app/blog/tcp-port-135/)
- **Port 137:** 'Phonebook' for older Windows networks. Matches computer names to IP addresses.
- **Port 445:** This port is for the SMB (Server Message Block) protocol. SMB allows devices on the same network to communicate with each other and share resources such as files and printers. (https://www.cbtnuggets.com/common-ports/what-is-port-445)

These ports can be exploited by hackers, especially when you are on a public network. Port 445 was the entry point for the infamous WannaCry ransomware via the EternalBlue exploit.